## Tutorial: How to Tune your Chameleon Hosts for Better Networking Performance

This notebook demonstrates how tune Linux hosts for networks. Much of this tutorial is based on the host tuning section of ESnet's [Faster Data](https://fasterdata.es.net/host-tuning/linux/) website.

The tutorial uses sharedwan1 ....

#### Modules 

- [Reserve a Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)



## Tutorial: 

#### Configure the Environment

In [10]:
import json
import os
import chi

from chi.server import *
from chi.lease import *
from chi.network import *

from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

#Config with your project and site
chi.set('project_name', 'CH-822154') # Replace with your project name

#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")+"hosts"
uc_server_name = username+'ServerUC'
tacc_server_name = username+'ServerTACC'
uc_lease_name = username+'LeaseUC'
tacc_lease_name = username+'LeaseTACC'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_skylake"

#### Create UC Lease

In [3]:
node_type="compute_haswell"

#Set site to UC
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=120)).strftime(BLAZAR_TIME_FORMAT)

server_count=1
fip_count=1

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_fip_reservation(reservation_list, count=fip_count)

# Create the lease
chi.blazar().lease.create(name=uc_lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
uc_lease = get_lease(get_lease_id(uc_lease_name))
    
#Print the lease info
print(json.dumps(uc_lease, indent=2))

uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

print("compute_reservation_id: " + uc_compute_reservation_id)
print("floatingip_reservation_id: " + uc_floatingip_reservation_id)

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org
{
  "status": "PENDING",
  "user_id": "02276834a17c4ca205d49eaa8c984eeaefc24c153f8a62bc16e5f880e7cce607",
  "name": "pruthhostsLeaseUC",
  "end_date": "2021-02-17T20:18:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "d80d7bd7-154d-40de-a9af-7cbe8076a5b8",
      "resource_id": "58295f8e-6b7c-49b4-bef4-490cd4f81039",
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "created_at": "2021-02-17 18:18:29",
      "updated_at": "2021-02-17 18:18:30",
      "required_floatingips": [],
      "missing_resources": false,
      "amount": 1,
      "id": "28a6af02-238c-46b2-9293-f9f7221af255",
      "resource_type": "virtual:floatingip",
      "resources_changed": false
    },
    {
      "status": "pending",
      "before_end": "default",
      "lease_id": "d80d7bd7-154d-40de-a9af-7cbe8076a5b

#### Create TACC Lease

In [5]:
node_type="compute_haswell"

#Set site to UC
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

server_count=1
fip_count=1

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
#add_fip_reservation(reservation_list, count=fip_count)

# Create the lease
chi.blazar().lease.create(name=tacc_lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

In [6]:
#Get the lease by name
tacc_lease = get_lease(get_lease_id(tacc_lease_name))
    
#Print the lease info
#print(json.dumps(tacc_lease, indent=2))

tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
#tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

print("compute_reservation_id: " + tacc_compute_reservation_id)
#print("floatingip_reservation_id: " + tacc_floatingip_reservation_id)

compute_reservation_id: 2e9a1254-8e12-434b-9053-64c8689c5f36


#### Configute the Network

In [ ]:
network = get_network_by_name('network_name')
network_id = network['id']

print('Network ID: ' + str(network_id))

In [ ]:
subnet = create_subnet(subnet_name, network_id, cidr=DEFAULT_CIDR, gateway_ip=None)
router = create_router(router_name, gw_network_name='public')
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

#### Start the Server

In [11]:
chi.use_site('CHI@UC')  
network_name='pruthNet'
compute_reservation_id=uc_compute_reservation_id
server_name=uc_server_name
#create the server
server = create_server(server_name, 
                       compute_reservation_id, 
                       key_name=key_name, 
                       network_id=network_id, 
                       network_name=network_name, 
                       nics=[], 
                       image_id=None, 
                       image_name=image_name, 
                       flavor_id=None, 
                       flavor_name=flavor_name, 
                       count=1)

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


In [12]:
chi.use_site('CHI@TACC')  
network_name='pruthNet'
compute_reservation_id=tacc_compute_reservation_id
server_name=tacc_server_name
#create the server
server = create_server(server_name, 
                       compute_reservation_id, 
                       key_name=key_name, 
                       network_id=network_id, 
                       network_name=network_name, 
                       nics=[], 
                       image_id=None, 
                       image_name=image_name, 
                       flavor_id=None, 
                       flavor_name=flavor_name, 
                       count=1)

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


#### Associate the Floating IP   (TODO: need to find floating_ip from the reservation that was just made)

In [ ]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

#### Delete Server

In [ ]:
delete_server(server['id'])

#### De-configure Network

In [ ]:
try:
    result = remove_subnet_from_router(router['id'], subnet['id'])
except Exception as e:
    print("remove_subnet_from_router error:" + str(e))
    pass

try:
    result = delete_router(router['id'])
except Exception as e:
    print("delete_router error: " + str(e))
    pass

try:
    result = delete_subnet(subnet['id'])
except Exception as e:
    print("delete_subnet error: " + str(e))
    pass

try:
    result = delete_network(network_id)
except Exception as e:
    print("delete_network error: " + str(e))
    pass

#### Release Lease

In [ ]:
delete_lease(lease_name)